# Zmiana formatu na użytkownik/produkt/ocena

In [2]:
import pandas as pd

In [3]:
file_path = 'preferencje.xlsx'
excel_data = pd.ExcelFile(file_path)
excel_data.sheet_names

['Arkusz1']

In [24]:
sheet_data = pd.read_excel(file_path, sheet_name='Arkusz1')
sheet_data.head(5)

,Unnamed: 0,https://hiszpanskiesmaki.pl/przepis-na-paella-mixta-z-owocami-morza-i-miesem/,https://www.kwestiasmaku.com/przepis/pascha-wielkanocna,https://aniagotuje.pl/przepis/rosol-z-kury,https://www.kwestiasmaku.com/przepis/zupa-z-dyni,https://www.kwestiasmaku.com/przepis/salatka-gyros,https://aniagotuje.pl/przepis/poledwiczki-w-sosie-musztardowym,Kaszanka z grilla | AniaGotuje.pl,Sałatka z ananasem | AniaGotuje.pl,https://www.mniammniam.com/przepis-na-miesny-jez,...,https://aniagotuje.pl/przepis/kebab,https://aniagotuje.pl/przepis/tatar,https://aniagotuje.pl/przepis/falafel,https://aniagotuje.pl/przepis/tradycyjny-barszcz-wigilijny,https://aniagotuje.pl/przepis/pierogi-ruskie,https://www.doradcasmaku.pl/przepis-kiszka-ziemniaczana-258674,https://aniagotuje.pl/przepis/strogonow,https://aniagotuje.pl/przepis/nalesniki-z-serem-i-owocami,https://www.kwestiasmaku.com/kuchnia_polska/pierogi/pierogi_z_miesem/przepis.html,https://www.digminecraft.com/food_recipes/make_steak.php
0,NaN,paella,pascha wielkanocna,rosół z kury,zupa dyniowa,Gyros,Polędwiczki w sosie musztardowym,Kaszanka,Sałatka z ananasem,Mięsny jeż,...,kebab,tatar,falafel,barszcz czerwony,pierogi ruskie,kiszka ziemniaczana,Boeuf strogonow,Naleśniki z serem,Pierogi z mięsem,Stek
1,Users/Items,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ula,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109815,3,3,5,5,4,3,2,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109521,4,5,4,3,5,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data = pd.read_excel(file_path, sheet_name='Arkusz1', skiprows=1) 

# Usuwanie pustych kolumn
data_cleaned = data.dropna(how="all", axis=1).dropna(how="all", axis=0)
data_cleaned.columns = ["user"] + list(data_cleaned.columns[1:])
# Nazwanie kolumn
data_melted = pd.melt(data_cleaned, id_vars=["user"], var_name="product", value_name="rating")
# Usunięcie pustych ratingów
data_final = data_melted.dropna(subset=["rating"])

data_final

,user,product,rating
1,Ula,paella,4.0
2,109815,paella,3.0
3,109521,paella,4.0
4,107792,paella,3.0
6,109812,paella,4.0
...,...,...,...
460,107791,Pierogi z mięsem,5.0
461,109550,Pierogi z mięsem,5.0
471,109794,Stek,5.0
477,109531,Stek,5.0


In [6]:
# Zapisanie pliku zgodnego z formatem biblioteki Surprise
data_final.to_csv("data.csv",index=False)

# Ocena algorytmów

In [32]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic, BaselineOnly, NormalPredictor
from surprise.model_selection import cross_validate


data = pd.read_csv("data.csv")  # Upewnij się, że plik zawiera odpowiednie kolumny
reader = Reader(rating_scale=(1.0, 5.0))  # Ustaw zakres ocen, np. od 1 do 5
dataset = Dataset.load_from_df(data[["user", "product", "rating"]], reader)

algorithms = {
    "BaselineOnly": BaselineOnly(),
        "BaselineOnly Params": BaselineOnly(bsl_options={
        "method": "sgd",
        "n_epochs": 20,
        "reg_u": 15,
        "reg_i": 10,
        "learning_rate": 0.005
    }),
    "SVD": SVD(),
    "SVD Params": SVD(n_factors=20, n_epochs=30, lr_all=0.005, reg_all=0.02),
    "Normal Predictor": NormalPredictor(),
    "KNN Basic": KNNBasic()


}

# Testowanie każdego algorytmu
results = {}
for name, algo in algorithms.items():
    print(f"Testowanie algorytmu: {name}")
    cv_results = cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=True)
    results[name] = cv_results

# Podsumowanie wyników
print("\nPodsumowanie wyników:")
for name, cv_results in results.items():
    print(f"\nAlgorytm: {name}")
    print(f"Średni RMSE: {cv_results['test_rmse'].mean():.4f}")


Testowanie algorytmu: BaselineOnly
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2447  1.3860  1.2130  1.2409  1.2985  1.2766  0.0613  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Testowanie algorytmu: BaselineOnly Params
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2428  1.3221  1.2473  1.3725  1.3827  1.3135  0.0595  
Fit time          0.00    0.00    0.00    0.00 

# Rekomendacja osobista

In [33]:
from surprise import Dataset, Reader, BaselineOnly, KNNBasic, SVD, NormalPredictor
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))  # Określenie zakresu ocen
data_surprise = Dataset.load_from_df(data[["user", "product", "rating"]], reader)

# Podział danych na zbiór treningowy i testowy
trainset, testset = train_test_split(data_surprise, test_size=0.2)

# Wybrany użytkownik
selected_user = '107885'  # Zamień na ID użytkownika, którego rekomendacje chcesz wygenerować

# Lista wszystkich produktów
all_products = data['product'].unique()

# Produkty, które użytkownik już ocenił
rated_products = data[data['user'] == selected_user]['product'].values

# Produkty, których użytkownik jeszcze nie ocenił
unrated_products = [p for p in all_products if p not in rated_products]

# Testowanie każdego algorytmu
for algo_name, algo in algorithms.items():
    print(f"\nRekomendacje dla użytkownika {selected_user} z algorytmem: {algo_name}")
    algo.fit(trainset)

    # Przewidywanie ocen dla nieocenionych produktów
    recommendations = []
    for product in unrated_products:
        prediction = algo.predict(selected_user, product)
        recommendations.append((product, prediction.est))

    # Sortowanie rekomendacji według przewidywanej oceny (malejąco)
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Wyświetlanie trzech najlepszych rekomendacji
    print(f"Trzy najbardziej rekomendowane potrawy dla {selected_user}:")
    for i in range(min(3, len(recommendations))):
        product, rating = recommendations[i]
        print(f"{i + 1}. {product}, ocena {rating:.2f}")



Rekomendacje dla użytkownika 107885 z algorytmem: BaselineOnly
Estimating biases using als...
Trzy najbardziej rekomendowane potrawy dla 107885:
1. kebab, ocena 3.99
2. ciasto murzynek, ocena 3.96
3. rosół z kury, ocena 3.91

Rekomendacje dla użytkownika 107885 z algorytmem: BaselineOnly Params
Estimating biases using sgd...
Trzy najbardziej rekomendowane potrawy dla 107885:
1. kebab, ocena 4.11
2. ciasto murzynek, ocena 4.11
3. rosół z kury, ocena 4.05

Rekomendacje dla użytkownika 107885 z algorytmem: SVD
Trzy najbardziej rekomendowane potrawy dla 107885:
1. ciasto murzynek, ocena 4.08
2. kebab, ocena 4.03
3. rosół z kury, ocena 3.98

Rekomendacje dla użytkownika 107885 z algorytmem: SVD Params
Trzy najbardziej rekomendowane potrawy dla 107885:
1. kebab, ocena 4.31
2. ciasto murzynek, ocena 4.18
3. rosół z kury, ocena 4.12

Rekomendacje dla użytkownika 107885 z algorytmem: Normal Predictor
Trzy najbardziej rekomendowane potrawy dla 107885:
1. paella , ocena 5.00
2. Stek, ocena 5.00


### Subiektywna ocena
###### SVD z dodatkowymi parametrami wskazało najdokładniej potrawy, które potencjalnie zamówiłbym do jedzenia, są one bardzo dobrze zarekomendowae. Wersje algorytmu BaselineOnly czysta oraz ze zmodyfikowanymi parametrami również trafnie przewiduje rekomendacje, lecz mają one niższe wagi niż w przypadku algorytmu SVD. Odwrotnie sytuacja wygląda dla funkcji NormalPredictor oraz KNN basic, gdzie żadne z wymienionych 3 dań nie wzbudza moich kulinarnych uniesień. 

### Wnioski
### Im niższa wartość RMSE, tym lepsza skuteczność algorytmu, ponieważ oznacza mniejszy błąd średniokwadratowy między przewidywaną oceną a rzeczywistą oceną.

Metoda / Wynik / Opis
SVD	1.3147	Metoda dekompozycji macierzy (Singular Value Decomposition), która modeluje ukryte czynniki w interakcjach użytkownik-produkt.
SVD Params	1.2488	Ulepszona wersja SVD z ręcznie dobranymi parametrami, co pozwoliło uzyskać najlepszy wynik RMSE.
BaselineOnly	1.2766	Algorytm wykorzystuje średnie oceny użytkowników i produktów do przewidywania ocen. Prosty i szybki.
BaselineOnly Params	1.3135	Wersja BaselineOnly z dostosowanymi parametrami, co może poprawić dopasowanie, ale czasem prowadzi do gorszych wyników.
KNN Basic	1.3200	Algorytm oparty na metodzie najbliższych sąsiadów (K-Nearest Neighbors), analizujący podobieństwo między użytkownikami lub produktami.
Normal Predictor	1.8202	Losowy algorytm przewidujący oceny jako losowe wartości w zakresie średniej i odchylenia standardowego.


Najlepszy wynik RMSE uzyskał algorytm SVD Params (1.2488), co sugeruje, że tuning parametrów dekompozycji macierzy pozwala osiągnąć dobre wyniki w przewidywaniu ocen.
BaselineOnly (1.2766) oraz KNN Basic (1.3200) również osiągnęły przyzwoite wyniki, pokazując, że prostsze algorytmy mogą być skuteczne.
Dostosowywanie parametrów (np. w SVD Params) może znacząco wpłynąć na efektywność algorytmu, poprawiając wyniki przewidywania.
Normal Predictor (1.8202) wykazał najsłabszy wynik, co jest oczekiwane, ponieważ jest to algorytm generujący losowe wyniki.


